# 7並べAI - オンライン学習版（Google Colab対応）

このノートブックは**自己進化するAI**です。6000試合を通じて自動的に強くなります。

## 特徴
- 🧠 **永続的記憶**: 試合をまたいで学習
- 📈 **オンライン学習**: 勝利した戦略を強化
- 🎯 **相手プロファイリング**: 相手の癖を記憶
- 🚀 **GPU対応**: CuPyで高速化可能

## 使い方
1. すべてのセルを順番に実行
2. 6000試合テストのセルを実行（約2-3時間）
3. 学習の進捗が10ゲームごとに表示されます

In [ ]:
# 環境セットアップ
import random
import copy
import time
from enum import Enum
from random import shuffle
import numpy as np

# GPU使用の試行（CuPyがあれば使用）
try:
    import cupy as cp
    USE_GPU = True
    print("✓ CuPy検出: GPU高速化を有効化")
    xp = cp
except ImportError:
    USE_GPU = False
    print("ℹ CuPyなし: CPUモードで実行")
    xp = np

print(f"\nモード: {'GPU (CuPy)' if USE_GPU else 'CPU (NumPy)'}")

In [ ]:
# 設定パラメータ
MY_PLAYER_NUM = 0  # 自分のプレイヤー番号

# シミュレーション設定
SIMULATION_COUNT = 700  # 精度と速度のバランス（Colab用に調整可能）
SIMULATION_DEPTH = 350

# オンライン学習のパラメータ
LEARNING_RATE = 0.05  # 学習率
WEIGHT_NOISE_STDDEV = 0.1  # ノイズ標準偏差
ENABLE_ONLINE_LEARNING = True  # オンライン学習を有効化

# 戦略パラメータ
ENABLE_TUNNEL_LOCK = True
ENABLE_BURST_FORCE = True
ENABLE_ADVANCED_HEURISTIC = True
ENABLE_NECROMANCER = True
ENABLE_SEVEN_SIGNAL = True
ENABLE_HYPERLOOP_DIST = True

# その他のパラメータ
BELIEF_STATE_DECAY_FACTOR = 0.05
DETERMINIZATION_ATTEMPTS = 120
STRATEGY_WEIGHT_MULTIPLIER = 1.5
TUNNEL_LOCK_WEIGHT = 3.5
BURST_FORCE_WEIGHT = 3.5
AGGRESSIVE_MODE_THRESHOLD = 0.6
DEFENSIVE_MODE_THRESHOLD = 0.8
AGGRESSIVENESS_MULTIPLIER = 0.3
URGENCY_MULTIPLIER = 0.5

# ヒューリスティックパラメータ
ACE_KING_BASE_BONUS = 10
TUNNEL_COMPLETE_BONUS = 10
ADJACENT_CARD_BONUS = 6
ADJACENT_CARD_PENALTY = 6
SAFE_MOVE_BONUS = 20
SUIT_CONCENTRATION_MULTIPLIER = 2.5
HAND_REDUCTION_BONUS = 0.15
CHAIN_POTENTIAL_MULTIPLIER = 15

# ロールアウトパラメータ
ROLLOUT_ACE_KING_BONUS = 10
ROLLOUT_ADJACENT_BONUS = 6
ROLLOUT_ADJACENT_PENALTY = 6
ROLLOUT_SAFE_BONUS = 12
ROLLOUT_SUIT_MULTIPLIER = 2.0
ROLLOUT_HAND_REDUCTION = 0.3
ROLLOUT_CHAIN_MULTIPLIER = 8

# 高度なヒューリスティックパラメータ
ADVANCED_HEURISTIC_PARAMS = {
    'W_CIRCULAR_DIST': 22,
    'W_MY_PATH': 112,
    'W_OTHERS_RISK': -127,
    'W_SUIT_DOM': 84,
    'W_WIN_DASH': 41,
    'P_THRESHOLD_BASE': 200,
    'P_KILL_ZONE': 300,
    'P_WIN_THRESHOLD': -31,
    'W_NECROMANCER': 20.0,
    'W_SEVEN_ADJACENT': 5.0,
    'W_SEVEN_NO_ADJ': -5.0,
}

ADVANCED_HEURISTIC_WEIGHT = 0.8

print("✓ パラメータ設定完了")
print(f"  - シミュレーション回数: {SIMULATION_COUNT}")
print(f"  - オンライン学習: {'有効' if ENABLE_ONLINE_LEARNING else '無効'}")
print(f"  - GPU: {'有効' if USE_GPU else '無効'}")

## ゲームエンジン（基本クラス）

In [ ]:
# 基本データクラス
class Suit(Enum):
    SPADE = '♠'
    CLUB = '♣'
    HEART = '♡'
    DIAMOND = '♢'
    def __str__(self):
        return self.value
    def __repr__(self):
        return f"Suit.{self.name}"

class Number(Enum):
    ACE = (1, 'A')
    TWO = (2, '2')
    THREE = (3, '3')
    FOUR = (4, '4')
    FIVE = (5, '5')
    SIX = (6, '6')
    SEVEN = (7, '7')
    EIGHT = (8, '8')
    NINE = (9, '9')
    TEN = (10, '10')
    JACK = (11, 'J')
    QUEEN = (12, 'Q')
    KING = (13, 'K')
    def __init__(self, val, label):
        self.val = val
        self.label = label
    def __str__(self):
        return self.label
    def __repr__(self):
        return f"Number.{self.name}"

class Card:
    def __init__(self, suit, number):
        self.suit = suit
        self.number = number
    def __str__(self):
        return f"{self.suit}{self.number}"
    def __repr__(self):
        return f"Card({self.suit!r}, {self.number!r})"
    def __eq__(self, other):
        if not isinstance(other, Card):
            return False
        return self.suit == other.suit and self.number == other.number
    def __hash__(self):
        return hash((self.suit, self.number))

class Hand(list):
    def check_number(self):
        return [card.number for card in self]
    def check_suit(self):
        return [card.suit for card in self]
    def choice(self, number, suit):
        for card in self:
            if card.number == number and card.suit == suit:
                return card
        return None
    def check(self, card):
        return card in self

class Deck(list):
    def __init__(self):
        super().__init__()
        for suit in Suit:
            for number in Number:
                self.append(Card(suit, number))
    def shuffle(self):
        shuffle(self)
    def draw(self):
        if len(self) > 0:
            return self.pop(0)
        return None
    def deal(self, players_num):
        hands = [Hand() for _ in range(players_num)]
        for i, card in enumerate(self):
            hands[i % players_num].append(card)
        return hands

print("✓ 基本クラス定義完了")

In [ ]:
# Stateクラス（ゲーム状態管理）
class State:
    def __init__(self, players_num=3):
        self.players_num = players_num
        deck = Deck()
        deck.shuffle()
        self.players_cards = deck.deal(players_num)
        
        # 場のカード（4スート × 13枚）
        self.field_cards = xp.zeros((4, 13), dtype=xp.int8)
        
        # 7を最初に配置
        for i in range(4):
            self.field_cards[i][6] = 1
        
        # 7を手札から削除
        for hand in self.players_cards:
            to_remove = []
            for card in hand:
                if card.number == Number.SEVEN:
                    to_remove.append(card)
            for card in to_remove:
                hand.remove(card)
        
        self.turn_player = 0
        self.pass_count = [0] * players_num
        self.out_player = []
        self.history = []  # (player, action, pass_flag)
    
    def my_hands(self):
        return self.players_cards[self.turn_player]
    
    def legal_actions(self):
        """場で出せるカードのリスト（トンネルルール対応）"""
        legal = []
        suit_to_index = {Suit.SPADE: 0, Suit.CLUB: 1, Suit.HEART: 2, Suit.DIAMOND: 3}
        
        for suit in Suit:
            suit_idx = suit_to_index[suit]
            field_row = self.field_cards[suit_idx]
            
            if USE_GPU:
                field_row = cp.asnumpy(field_row)
            
            # 左端を探す
            left_end = -1
            for i in range(13):
                if field_row[i] == 1:
                    left_end = i
                    break
            
            # 右端を探す
            right_end = -1
            for i in range(12, -1, -1):
                if field_row[i] == 1:
                    right_end = i
                    break
            
            # トンネルルール判定
            if left_end == 0 and right_end < 12:
                # Aが出ている→K側のみ伸ばせる
                if right_end + 1 <= 12:
                    legal.append(Card(suit, list(Number)[right_end + 1]))
            elif right_end == 12 and left_end > 0:
                # Kが出ている→A側のみ伸ばせる
                if left_end - 1 >= 0:
                    legal.append(Card(suit, list(Number)[left_end - 1]))
            else:
                # 通常: 両側に伸ばせる
                if left_end > 0:
                    legal.append(Card(suit, list(Number)[left_end - 1]))
                if right_end < 12:
                    legal.append(Card(suit, list(Number)[right_end + 1]))
        
        return legal
    
    def my_actions(self):
        """現在のプレイヤーが出せるカード"""
        legal = self.legal_actions()
        my_hand = self.my_hands()
        return [card for card in legal if card in my_hand]
    
    def is_done(self):
        """ゲーム終了判定"""
        # 誰かの手札が0
        for i, hand in enumerate(self.players_cards):
            if len(hand) == 0 and i not in self.out_player:
                return True
        # 全員バースト
        if len(self.out_player) >= self.players_num - 1:
            return True
        return False
    
    def next(self, action, pass_flag=0):
        """状態を更新"""
        current = self.turn_player
        
        # 履歴に記録
        self.history.append((current, action, pass_flag))
        
        if pass_flag == 1:
            self.pass_count[current] += 1
            if self.pass_count[current] >= 3:
                self.out_player.append(current)
                # 手札を場に出す
                suit_to_index = {Suit.SPADE: 0, Suit.CLUB: 1, Suit.HEART: 2, Suit.DIAMOND: 3}
                for card in self.players_cards[current]:
                    suit_idx = suit_to_index[card.suit]
                    num_idx = card.number.val - 1
                    self.field_cards[suit_idx][num_idx] = 1
                self.players_cards[current] = Hand()
        else:
            if action is not None:
                # カードを場に出す
                suit_to_index = {Suit.SPADE: 0, Suit.CLUB: 1, Suit.HEART: 2, Suit.DIAMOND: 3}
                suit_idx = suit_to_index[action.suit]
                num_idx = action.number.val - 1
                self.field_cards[suit_idx][num_idx] = 1
                # 手札から削除
                self.players_cards[current].remove(action)
        
        # 次のプレイヤー
        self.turn_player = (current + 1) % self.players_num
        while self.turn_player in self.out_player:
            self.turn_player = (self.turn_player + 1) % self.players_num
        
        return self
    
    def clone(self):
        """状態の深いコピー"""
        new_state = State.__new__(State)
        new_state.players_num = self.players_num
        new_state.players_cards = [Hand(list(hand)) for hand in self.players_cards]
        new_state.field_cards = self.field_cards.copy()
        new_state.turn_player = self.turn_player
        new_state.pass_count = list(self.pass_count)
        new_state.out_player = list(self.out_player)
        new_state.history = list(self.history)
        return new_state

print("✓ Stateクラス定義完了")

## AI実装（学習機能付き）

In [ ]:
# CardTracker（推論エンジン）
class CardTracker:
    """相手の手札を推論するクラス"""
    def __init__(self, state, my_player_num):
        self.my_player_num = my_player_num
        self.players_num = state.players_num
        
        # 各プレイヤーが持ちうるカードの集合
        all_cards = set()
        for suit in Suit:
            for number in Number:
                all_cards.add(Card(suit, number))
        
        # 自分のカードと場のカードを除外
        my_cards = set(state.players_cards[my_player_num])
        field_cards_set = set()
        suit_to_index = {Suit.SPADE: 0, Suit.CLUB: 1, Suit.HEART: 2, Suit.DIAMOND: 3}
        for suit in Suit:
            suit_idx = suit_to_index[suit]
            field_row = state.field_cards[suit_idx]
            if USE_GPU:
                field_row = cp.asnumpy(field_row)
            for i, val in enumerate(field_row):
                if val == 1:
                    field_cards_set.add(Card(suit, list(Number)[i]))
        
        unknown_cards = all_cards - my_cards - field_cards_set
        
        # 初期状態：自分以外のプレイヤーが持ちうる
        self.possible = {}
        for p in range(self.players_num):
            if p == my_player_num:
                self.possible[p] = my_cards
            else:
                self.possible[p] = set(unknown_cards)
    
    def observe_action(self, player, action, is_pass, state):
        """行動を観測して推論を更新"""
        if is_pass:
            # パス→その時出せるカードを持っていない
            legal = state.legal_actions()
            for card in legal:
                if card in self.possible[player]:
                    self.possible[player].discard(card)
        elif action is not None:
            # カードをプレイ→全員がそのカードを持たない
            for p in range(self.players_num):
                if action in self.possible[p]:
                    self.possible[p].discard(action)

print("✓ CardTracker定義完了")

In [ ]:
# OpponentModel（相手プロファイリング）
class OpponentModel:
    """相手の行動パターンを分析（永続的記憶付き）"""
    
    # クラス変数（永続的記憶）
    _persistent_opponent_data = {}
    _game_count = 0
    
    # 定数
    MOVING_AVERAGE_ALPHA = 0.3
    PASS_RATE_THRESHOLD = 0.2
    ACE_KING_RATE_THRESHOLD = 0.3
    
    def __init__(self, players_num):
        self.players_num = players_num
        self.flags = {p: {
            "aggressive": 0,
            "blocker": 0,
            "tunnel_usage": 0,
            "pass_count": 0,
            "end_cards": 0,
            "middle_cards": 0,
            "total_actions": 0,
        } for p in range(players_num)}
        
        self.turn_count = 0
        
        # 永続データの初期化
        for p in range(players_num):
            if p not in OpponentModel._persistent_opponent_data:
                OpponentModel._persistent_opponent_data[p] = {
                    'pass_rate': 0.0,
                    'ace_king_immediate_rate': 0.0,
                    'tunnel_usage_rate': 0.0,
                    'game_count': 0,
                }
    
    def observe(self, state, player, action, pass_flag):
        """行動観測"""
        self.turn_count += 1
        
        if pass_flag == 1:
            self.flags[player]["pass_count"] += 1
            self.flags[player]["blocker"] += 1
            return
        
        if action is None or isinstance(action, list):
            return
        
        self.flags[player]["total_actions"] += 1
        
        if action.number in (Number.ACE, Number.KING):
            self.flags[player]["aggressive"] += 2
            self.flags[player]["end_cards"] += 1
            
            suit_idx = list(Suit).index(action.suit)
            field_row = state.field_cards[suit_idx]
            if USE_GPU:
                field_row = cp.asnumpy(field_row)
            
            if action.number == Number.ACE:
                if field_row[12] == 0:
                    self.flags[player]["tunnel_usage"] += 1
            elif action.number == Number.KING:
                if field_row[0] == 0:
                    self.flags[player]["tunnel_usage"] += 1
        
        if action.number in (Number.SIX, Number.EIGHT):
            self.flags[player]["middle_cards"] += 1
            self.flags[player]["blocker"] += 0.5
        
        if action.number in (Number.JACK, Number.QUEEN):
            self.flags[player]["blocker"] += 1
    
    def update_persistent_stats(self, player):
        """ゲーム終了時に永続データを更新"""
        flags = self.flags[player]
        persistent = OpponentModel._persistent_opponent_data[player]
        
        total_turns = flags["pass_count"] + flags["total_actions"]
        if total_turns == 0:
            return
        
        current_pass_rate = flags["pass_count"] / total_turns
        safe_total_actions = max(1, flags["total_actions"])
        current_ace_king_rate = flags["end_cards"] / safe_total_actions
        current_tunnel_rate = flags["tunnel_usage"] / safe_total_actions
        
        alpha = OpponentModel.MOVING_AVERAGE_ALPHA
        persistent['pass_rate'] = (1 - alpha) * persistent['pass_rate'] + alpha * current_pass_rate
        persistent['ace_king_immediate_rate'] = (1 - alpha) * persistent['ace_king_immediate_rate'] + alpha * current_ace_king_rate
        persistent['tunnel_usage_rate'] = (1 - alpha) * persistent['tunnel_usage_rate'] + alpha * current_tunnel_rate
        persistent['game_count'] += 1
    
    def get_persistent_stats(self, player):
        if player in OpponentModel._persistent_opponent_data:
            return OpponentModel._persistent_opponent_data[player]
        return None
    
    def mode(self, player):
        """戦略モード判定"""
        flags = self.flags[player]
        a = flags["aggressive"]
        b = flags["blocker"]
        pass_count = flags["pass_count"]
        tunnel_usage = flags["tunnel_usage"]
        
        persistent = self.get_persistent_stats(player)
        if persistent and persistent['game_count'] > 0:
            persistent_pass_tendency = persistent['pass_rate'] > OpponentModel.PASS_RATE_THRESHOLD
            persistent_ace_king_tendency = persistent['ace_king_immediate_rate'] > OpponentModel.ACE_KING_RATE_THRESHOLD
            
            if persistent_pass_tendency and pass_count >= 1:
                return "burst_force"
            if persistent_ace_king_tendency and tunnel_usage >= 1:
                return "tunnel_lock"
        
        if pass_count >= 2:
            return "burst_force"
        
        if tunnel_usage >= 2 or a >= b + 3:
            return "tunnel_lock"
        
        if b >= a + 2:
            return "burst_force"
        
        return "neutral"
    
    def get_threat_level(self, player):
        """脅威度計算"""
        flags = self.flags[player]
        threat = 0.5
        threat += flags["aggressive"] * 0.05
        threat += flags["tunnel_usage"] * 0.1
        threat -= flags["pass_count"] * 0.1
        
        persistent = self.get_persistent_stats(player)
        if persistent and persistent['game_count'] > 0:
            threat += (1.0 - persistent['pass_rate']) * 0.15
            threat += persistent['ace_king_immediate_rate'] * 0.2
        
        return max(0.0, min(1.0, threat))

print("✓ OpponentModel定義完了")

## メインAI（HybridStrongestAI with Learning）

このセルは実行に時間がかかるため、コードのみ表示します。

In [ ]:
# このファイルは大きいため、src/main.pyから必要な部分をインポートする形で実装
# 実際の提出時は、src/main.pyのHybridStrongestAIクラス全体をここに貼り付けてください

print("⚠ 注意: HybridStrongestAIクラスの完全な実装が必要です")
print("   src/main.pyのHybridStrongestAIクラス（約600行）をここにコピーしてください")
print("\nここでは簡易版を定義します（テスト用）")

# 簡易版（実際はsrc/main.pyの完全版を使用）
class HybridStrongestAI:
    _best_weights = None
    _trial_weights = None
    _game_results = []
    _total_games = 0
    _wins = 0
    MAX_GAME_RESULTS_HISTORY = 100
    
    def __init__(self, my_player_num, simulation_count=50):
        self.my_player_num = my_player_num
        self.simulation_count = simulation_count
        self._opponent_model = None
        self._in_simulation = False
        
        if HybridStrongestAI._best_weights is None:
            HybridStrongestAI._best_weights = self._initialize_weights()
            HybridStrongestAI._trial_weights = self._initialize_weights()
    
    def _initialize_weights(self):
        return dict(ADVANCED_HEURISTIC_PARAMS)
    
    def get_action(self, state):
        # 簡易実装（実際はsrc/main.pyの完全版を使用）
        my_actions = state.my_actions()
        if not my_actions:
            return None, 1
        return random.choice(my_actions), 0
    
    def prepare_next_game(self):
        if ENABLE_ONLINE_LEARNING and HybridStrongestAI._best_weights:
            trial = {}
            for key, value in HybridStrongestAI._best_weights.items():
                noise_scale = max(abs(value), 10.0)
                noise = random.gauss(0, WEIGHT_NOISE_STDDEV) * noise_scale
                trial[key] = value + noise
            HybridStrongestAI._trial_weights = trial
    
    def update_weights_after_game(self, won):
        if not ENABLE_ONLINE_LEARNING:
            return
        
        HybridStrongestAI._total_games += 1
        if won:
            HybridStrongestAI._wins += 1
        
        HybridStrongestAI._game_results.append(1 if won else 0)
        if len(HybridStrongestAI._game_results) > HybridStrongestAI.MAX_GAME_RESULTS_HISTORY:
            HybridStrongestAI._game_results.pop(0)
        
        if won:
            for key in HybridStrongestAI._best_weights:
                best = HybridStrongestAI._best_weights[key]
                trial = HybridStrongestAI._trial_weights[key]
                HybridStrongestAI._best_weights[key] = best + LEARNING_RATE * (trial - best)
        
        if HybridStrongestAI._total_games % 10 == 0:
            win_rate = HybridStrongestAI._wins / HybridStrongestAI._total_games
            recent_games = min(10, len(HybridStrongestAI._game_results))
            recent_win_rate = sum(HybridStrongestAI._game_results[-recent_games:]) / recent_games
            print(f"[Learning] Games: {HybridStrongestAI._total_games}, "
                  f"Overall: {win_rate:.2%}, Recent ({recent_games}): {recent_win_rate:.2%}")
    
    def get_current_weights(self):
        if ENABLE_ONLINE_LEARNING:
            return HybridStrongestAI._trial_weights
        return ADVANCED_HEURISTIC_PARAMS

print("✓ HybridStrongestAI定義完了（簡易版）")
print("   ⚠ 実際の提出時は完全版を使用してください")

In [ ]:
# ランダムAI
def random_action(state):
    my_actions = state.my_actions()
    if my_actions:
        return random.choice(my_actions)
    return None

# AIインスタンス作成
ai_instance = HybridStrongestAI(MY_PLAYER_NUM, simulation_count=SIMULATION_COUNT)

def my_AI(state):
    return ai_instance.get_action(state)

print("✓ AI初期化完了")

## 6000試合テスト

このセルを実行すると6000試合のテストが開始されます。
- 所要時間: 約2-3時間（GPU使用時は短縮可能）
- 10ゲームごとに学習統計が表示されます
- 最終的な勝率が85-95%に達することを期待

In [ ]:
# 6000試合テスト
def run_6000_game_test():
    game_count = 6000
    wins = [0, 0, 0]
    ai_pos = MY_PLAYER_NUM
    
    print("="*60)
    print("6000試合テスト開始")
    print("="*60)
    print(f"AIプレイヤー: P{ai_pos}")
    print(f"シミュレーション回数: {SIMULATION_COUNT}")
    print(f"オンライン学習: {'有効' if ENABLE_ONLINE_LEARNING else '無効'}")
    print(f"GPU: {'有効 (CuPy)' if USE_GPU else '無効 (NumPy)'}")
    print("="*60)
    print()
    
    start_time = time.time()
    
    for i in range(game_count):
        # ゲーム準備
        ai_instance.prepare_next_game()
        state = State()
        
        # ゲーム実行
        while not state.is_done():
            current_player = state.turn_player
            
            if current_player == ai_pos:
                action, pass_flag = my_AI(state)
            else:
                action = random_action(state)
                pass_flag = 0
                if action is None:
                    pass_flag = 1
            
            state.next(action, pass_flag)
        
        # 勝者判定
        winner = -1
        for p, hand in enumerate(state.players_cards):
            if len(hand) == 0 and p not in state.out_player:
                winner = p
                break
        
        if winner == -1:
            survivors = [p for p in range(state.players_num) if p not in state.out_player]
            if len(survivors) == 1:
                winner = survivors[0]
        
        if winner != -1:
            wins[winner] += 1
        
        # 学習更新
        if ai_instance._opponent_model:
            for p in range(state.players_num):
                if p != ai_pos:
                    ai_instance._opponent_model.update_persistent_stats(p)
        
        won = (winner == ai_pos)
        ai_instance.update_weights_after_game(won)
        
        # 進捗表示（100ゲームごと）
        if (i + 1) % 100 == 0:
            elapsed = time.time() - start_time
            avg_time = elapsed / (i + 1)
            remaining_time = avg_time * (game_count - i - 1)
            current_win_rate = wins[ai_pos] / (i + 1) * 100
            
            print(f"進捗: {i + 1}/{game_count} | "
                  f"勝率: {current_win_rate:.1f}% | "
                  f"残り時間: {remaining_time/60:.1f}分")
    
    end_time = time.time()
    duration = end_time - start_time
    
    # 最終結果
    print()
    print("="*60)
    print("6000試合テスト完了")
    print("="*60)
    print(f"総実行時間: {duration/60:.1f}分 ({duration/3600:.2f}時間)")
    print(f"平均時間: {duration/game_count:.2f}秒/ゲーム")
    print()
    
    ai_wins = wins[ai_pos]
    ai_win_rate = ai_wins / game_count * 100
    print(f"🤖 AI最終勝率: {ai_wins}/{game_count} ({ai_win_rate:.2f}%)")
    print()
    
    print("各プレイヤーの成績:")
    for i, w in enumerate(wins):
        win_rate = w / game_count * 100
        label = "AI" if i == ai_pos else f"Random{i}"
        print(f"  P{i} ({label}): {w}/{game_count} ({win_rate:.2f}%)")
    
    # 評価
    print()
    if ai_win_rate >= 85:
        print("🏆 優秀！目標勝率85%以上を達成しました！")
    elif ai_win_rate >= 70:
        print("✓ 良好！ベースライン(33.3%)を大きく上回っています")
    elif ai_win_rate >= 50:
        print("✓ 合格！ベースライン(33.3%)を上回っています")
    else:
        print("⚠ 要改善: さらなるチューニングが必要です")
    
    return ai_win_rate

print("✓ テスト関数定義完了")
print("\n次のセルを実行して6000試合テストを開始してください")

In [ ]:
# 6000試合テスト実行
final_win_rate = run_6000_game_test()

## 結果の可視化

In [ ]:
# 学習曲線のプロット（オプション）
try:
    import matplotlib.pyplot as plt
    
    if HybridStrongestAI._game_results:
        # 移動平均を計算
        window = 100
        moving_avg = []
        for i in range(len(HybridStrongestAI._game_results)):
            start = max(0, i - window + 1)
            avg = sum(HybridStrongestAI._game_results[start:i+1]) / (i - start + 1)
            moving_avg.append(avg * 100)
        
        plt.figure(figsize=(12, 6))
        plt.plot(moving_avg, label='Win Rate (moving avg)')
        plt.axhline(y=85, color='g', linestyle='--', label='Target (85%)')
        plt.axhline(y=33.3, color='r', linestyle='--', label='Baseline (33.3%)')
        plt.xlabel('Games')
        plt.ylabel('Win Rate (%)')
        plt.title('Learning Curve - Win Rate Over 6000 Games')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
        print("✓ 学習曲線を表示しました")
    else:
        print("⚠ データなし: 6000試合テストを先に実行してください")
except ImportError:
    print("ℹ matplotlibがインストールされていません")
    print("  可視化をスキップします")

## 提出用: my_AI関数

大会提出時は、この関数をコピーして使用してください。

In [ ]:
# 提出用my_AI関数
# ※実際の提出時は、HybridStrongestAIクラスの完全版と一緒に提出してください

def my_AI(state):
    """7並べAI - オンライン学習版
    
    このAIは試合ごとに学習し、6000試合で85-95%の勝率を目指します。
    """
    return ai_instance.get_action(state)

print("✓ 提出用my_AI関数")
print("\n提出時の注意:")
print("1. HybridStrongestAIクラスの完全版を含めること")
print("2. OpponentModel, CardTrackerクラスも含めること")
print("3. ENABLE_ONLINE_LEARNING = True を確認")
print("4. すべてのパラメータ定義を含めること")